# Homework 03 Word Embeddings

### Goal 

The **goal** of this homework is to provide an opportunity to build an end-to-end system.

Specifically, we are going to build a word embedding system, that can 

1. Read and preprocess raw data
2. Use two different ways (latent semantic analysis and skip-gram) to learn word embeddings
3. Evaluate the quality of word embeddings using some intrinsic evaluation methods 

### Submission

Your submission should only include this notebook file. Please keep **all the outputs** of this notebook in your submission for grading. We will run the code only if we are not sure it is correct.

### Dependency

You will need the following package to finish this homework assignment

- [nltk](https://www.nltk.org) for word tokenization
- [fasttext](https://pypi.org/project/fasttext/) for the implementation of the Skip-gram model

### Hint

Search for the keyword `TODO` in this notebook to find out which parts need your input

## 0. Preparation

Please run the following code to download the data

In [1]:
# Download the data from course webpage
# import urllib.request
# from os.path import isfile
# if not isfile("embeddings/imdb-small.txt"):
#     url = "https://yangfengji.net/uva-nlp-course/data/embeddings.zip"
#     print("Downloading ...")
#     filename, headers = urllib.request.urlretrieve(url, filename="embeddings.zip")

#     print("Decompressing the file ...")
#     !unzip embeddings.zip

sents = open("embeddings/imdb-small.txt").read().split("\n")
print("Read {} sentences".format(len(sents)))

Read 10000 sentences


## 1. Data Processing (5 points)

Data processing is an **essential** skill for NLP researchers. Unlike machine learning where researchers sometimes may want to use synthetic data to demonstrate the potential of their algorithms, NLP researchers need to deal with real-world data all the time. Unfortunately, this means that these data are noisy and often contain irregular patterns. Therefore, a reasonable data processing can alleviate the challenge of building NLP systems to some extent and may also help boost the performance of machine learning models.

Data processing for learning word embeddings includes two basic modules

- Tokenizing texts and replacing some special tokens
- Filtering low-frequency and building a vocab

### 1.1 Tokenization (3 points)

The following function *tokenize()* should include the following components

1. Load the raw text from the file named **imdb-small.txt**
2. Convert all characters into lower cases
3. Tokenize the raw text using `nltk.tokenize` 
4. Remove all punctuation (as single tokens) and replace all numbers with a special token `<num>`
5. Write the preprocessed text to the file named **imdb-small.txt.tokenized** and maintain the same format (one paragraph per line)

(The file names are pre-defined, please do not change them.)

In [2]:
import nltk
nltk.download("punkt")
from nltk import RegexpTokenizer

def tokenize(infname="embeddings/imdb-small.txt"):
    # 1. Load the raw text from the file named imdb-small.txt
    sentences = open(infname).read().split("\n")
    outfname = open(infname + ".tokenized", "w")
    
    # ----------------------------------------
    for s, sent in enumerate(sentences):
        # 2. Convert all characters into lower cases
        lower_case_sent = sent.lower()

        # 3. Tokenize the raw text using nltk.tokenize
        # 4. Remove all punctuation (as single tokens)
        remove_punctuation = RegexpTokenizer(r"\w+")
        tokenized_sent = remove_punctuation.tokenize(lower_case_sent)

        # 4. Replace all numbers with a special token <num>
        for i, t in enumerate(tokenized_sent):
            if t.isnumeric():
                tokenized_sent[i] = "<num>"

        # 5. Write the preprocessed text to the file
        for t in tokenized_sent:
            outfname.write('%s ' %t)
        if s < 9999:
            outfname.write('\n')

    # ----------------------------------------
    outfname.close()

[nltk_data] Downloading package punkt to /home/ft/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.2 Filtering (2 points)

The following function *token_filter()* should include the following components

1. Remove the words that appear in the data less than 5 times (word_frequency < 5)
2. Write the filtered data to the file named **imdb-small.txt.filtered** and maintain the same format (one sentence per line)
3. Return a Python list that contains all the words

In [3]:
from nltk.lm import Vocabulary

def token_filter(infname="embeddings/imdb-small.txt.tokenized", thresh=5):
    tokenized_sentences = open(infname).read().split("\n")
    outfname = open(infname.replace(".tokenized", ".filtered"), 'w')
    
    # ----------------------------------------
    words = list()
    for t_sent in tokenized_sentences:
        for word in t_sent.split():
            words.append(word)

    # 1. Remove the words that appear in the data less than "thresh" times
    vocab = Vocabulary(words, unk_cutoff=5)

    # 2. Write the filtered data to the file named imdb-small.txt.filtered
    for t, tokenized_sent in enumerate(tokenized_sentences):
        filtered_sent = list()
        for word in tokenized_sent.split():
            # if vocab.lookup(word) != '<UNK>':
            filtered_sent.append(vocab.lookup(word).lower())
        
        for word in filtered_sent:
            outfname.write('%s ' %word)
        if t < 9999:
            outfname.write('\n')

    # ----------------------------------------
    outfname.close()

    # 3. Return a Python list that contains all the words
    return list(vocab)

### 1.3 Put all together

The following code block will call the previous two functions to do data preprocessing.

In [4]:
tokenize()
vocab = token_filter()
print("The vocab size = {}".format(len(vocab)))

The vocab size = 18323


## 2. Word Embeddings (5 points)

In this section, you need to implement two different ways of constructing word embeddings: latent semantic analysis  and skipgram. 

### 2.1 Latent semantic analysis (3 points)

The function of LSA should include the following components

- Construct the word-doc matrix using `CountVectorizer` with `tokenizer=lambda x : x.split()`, make sure in this matrix that each row represents one word and each column represents one document (sentence, to be accurate in this case)
- Use the `TruncatedSVD` from `sklearn.decomposition` to factorize the word-doc matrix
- Construct the word embedding matrix with dimensionality as $v \times k$, where $v$ is the vocab size and $k$ is the word embedding dimension

The LSA() function should return 

- **embeddings**: A matrix with size $v\times k$ that contains all the word embeddings
- **vocab**: A Python dict with size $v$ that maps a word to the corresponding word index. Please pay attention to the mapping relation in vocab, which will be needed in the evaluation section.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

def LSA(fname = "embeddings/imdb-small.txt.filtered", dim=50):
    sents = open(fname).read().split("\n")
    
    # 1. Construct the word-doc matrix using CountVectorizer
    vectorizer =  CountVectorizer(tokenizer=lambda x : x.split())
    word_doc = vectorizer.fit_transform(sents)
    word_doc = word_doc.reshape(word_doc.shape[1], word_doc.shape[0])

    # 2. Use TruncatedSVD to factorize the word-doc matrix
    svd = TruncatedSVD(n_components=50)

    # 3. Construct the word embedding matrix with dimensionality as v x k
    embeddings = svd.fit_transform(word_doc)
    
    # Vocabulary
    vocab = vectorizer.vocabulary_

    return embeddings, vocab

### 2.2 Skip-gram model (2 points)

In this section, you do not have to implement the skip-gram model by yourself. An authentic implementation of skip-gram can be found in the Python package [fasttext](https://pypi.org/project/fasttext/), which you can install on the your local machine with the folllwing commandline or directly load the package if you are using Google Colab. 
```python
pip install fasttext
```

In the following code, please use the `fasttext.train_unsupervised` function for the skipgram() implementation. For the `fasttext.train_unsupervised`, please use the following configurations

- `model='skipgram'`
- Context window size: `ws = 3` 
- Word embedding dimension: `dim = 50`
- Number of negative examples: `neg = 5`

For all other parameters, use their default values.

Similar to the previous `LSA()`, `Skipgram()` should return 

- **embeddings**: A matrix with size $v\times k$ that contains all the word embeddings
- **vocab**: A Python dict with size $v$ that maps an index to the corresponding word

To get the word embeddings and vocab from fasttext, you need to understand [some functions](https://pypi.org/project/fasttext/#api) provided by the `model` object in the fasttext.

In [6]:
from fasttext import train_unsupervised

def Skipgram(fname = "embeddings/imdb-small.txt.filtered", ws=3, dim=50):
    model = train_unsupervised("embeddings/imdb-small.txt.filtered", model='skipgram', ws=ws, dim=dim, neg=5)

    embeddings = model.get_output_matrix()
    
    v = model.get_words()
    vocab = dict()
    for i, v_word in enumerate(v):
        vocab[v_word] = i

    return embeddings, vocab

### 2.3 Put all together

Run the following code blocks to get word embeddings from two different methods. It may take a couple of minutes to compute both embeddings.

In [7]:
embeddings_lsa, vocab_lsa = LSA()
embeddings_sg, vocab_sg = Skipgram()

In [12]:
len(vocab_sg)

18324

The following code will serve as the sanity check that `vocab_lsa` and `vocab_sg` contain the same words

In [8]:
lsa_word_set = set([item[0] for item in vocab_lsa.items()])
sg_word_set = set([item[0] for item in vocab_sg.items()])
sym_diff = lsa_word_set.symmetric_difference(sg_word_set)

if len(sym_diff) == 0:
    print("vocab_lsa and vocab_sg contain the same words!")
else:
    print("The word that only appear in one vocab: {}".format(sym_diff))

The word that only appear in one vocab: {'</s>'}


If the only word from the `symmetric_difference()` function is `</s>`, then your implementation should be fine. (`</s>` was added by `fasttext` automatically to the end of each text.)

## 3. Evaluation (6 points)

In this homework, we will only use intrinsic evaluation. Specifically, for a list of predefined word pairs with their similarity scores, the evaluation is to calculate the correlation between the predefined similarity scores and the cosine similarity scores based on word embeddings. The higher the correlation, the better the quality of word embeddings.

In [9]:
def load_wordpairs(fname = "embeddings/word-pairs.txt", vocab=vocab):
    records = {}
    with open(fname) as fin:
        for line in fin:
            items = line.strip().split(",")
            if (items[1] in vocab) and (items[2] in vocab): # make sure both words in the vocab
                records[(items[1],items[2])] = float(items[3])
    print("Load {} pairs of words for evaluation".format(len(records)))
    return records

### 3.1 Word similarity correlation (3 points)

The purpose of this section is to implement the correlation function that compares the predefined scores and the scores computed by cosine similarity. The code of the correlation function is almost done, and the only thing left is the code for computing cosine similarity. 

In [14]:
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

def correlation(records, embeddings, vocab):
    predefined_scores = []
    cossim_scores = []
    for (words, sim_score) in records.items():
        predefined_scores.append(sim_score)

        word0 = embeddings[vocab[words[0]]]
        word1 = embeddings[vocab[words[1]]]

        cos_sim = cosine_similarity(word0.reshape(1,50),word1.reshape(1,50))
        cossim_scores.append(cos_sim[0][0])

    corr = pearsonr(predefined_scores, cossim_scores)
    return corr

Run the following code block to calculate the correlations between pre-defined similarity scores and the cosine similarity scores based on word embeddings

In [15]:
# Load records
records_lsa = load_wordpairs(vocab=vocab_lsa)
records_sg = load_wordpairs(vocab=vocab_sg)

Load 151 pairs of words for evaluation
Load 151 pairs of words for evaluation


In [16]:
corr_lsa = correlation(records_lsa, embeddings_lsa, vocab_lsa)
print("The correlation with the LSA embeddings = {} with p-value {}".format(corr_lsa[0], corr_lsa[1]))
corr_sg = correlation(records_sg, embeddings_sg, vocab_sg)
print("The correlation with the SG embeddings = {} with p-value {}".format(corr_sg[0], corr_sg[1]))

if corr_lsa[0] > corr_sg[0]:
    print("LSA is better than Skip-gram")
elif corr_lsa[0] < corr_sg[0]:
    print("Skipgram is better than LSA")

The correlation with the LSA embeddings = 0.010522503294 with p-value 0.8979655774734477
The correlation with the SG embeddings = 0.356725477141825 with p-value 6.927441041383524e-06
Skipgram is better than LSA


### 3.2 Analysis of context window size in Skipgram (3 points)

With the correlation function, we can analyze the effect of different context window sizes in the Skipgram model. Specifically, please call the previous implementation 

- `Skipgram(fname, ws, dim=50)` with the context window size `ws` as 3, 6, 9, 12, 15
- For each context window size, calculate the correlation using the function `correlation(records, embeddings, vocab)`
- **Print out** the fives correlation scores in your final submission: one score per line with the following format
<center> ws\t correlation</center>

In [17]:
for ws in [3,6,9,12,15]:
    embeddings_sg, vocab_sg = Skipgram(ws=ws)
    corr_sg = correlation(records_sg, embeddings_sg, vocab_sg)
    print("{}\t{}".format(ws,corr_sg[0]))

3	0.22480088770813567
6	0.2684002356121604
9	0.2867693641700646
12	0.373773999400796
15	0.30800427112237727


Similar experiment can also be conducted on the parameter of negative examples `neg`, but it will not be included in this homework.